In [2]:
import pandas as pd
import numpy as np
import jieba
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# 获取数据

In [11]:
data = pd.read_csv('./data/书籍评价.csv',encoding='gbk')
data

,Unnamed: 0,内容,评价
0,0,从编程小白的角度看，入门极佳。,好评
1,1,很好的入门书，简洁全面，适合小白。,好评
2,2,讲解全面，许多小细节都有顾及，三个小项目受益匪浅。,好评
3,3,前半部分讲概念深入浅出，要言不烦，很赞,好评
4,4,看了一遍还是不会写，有个概念而已,差评
5,5,中规中矩的教科书，零基础的看了依旧看不懂,差评
6,6,内容太浅显，个人认为不适合有其它语言编程基础的人,差评
7,7,破书一本,差评
8,8,适合完完全全的小白读，有其他语言经验的可以去看别的书,差评
9,9,基础知识写的挺好的！,好评


# 数据基本处理

In [13]:
content = data["内容"]
content

0                从编程小白的角度看，入门极佳。
1              很好的入门书，简洁全面，适合小白。
2      讲解全面，许多小细节都有顾及，三个小项目受益匪浅。
3            前半部分讲概念深入浅出，要言不烦，很赞
4               看了一遍还是不会写，有个概念而已
5           中规中矩的教科书，零基础的看了依旧看不懂
6       内容太浅显，个人认为不适合有其它语言编程基础的人
7                           破书一本
8     适合完完全全的小白读，有其他语言经验的可以去看别的书
9                     基础知识写的挺好的！
10                           太基础
11            略_嗦。。适合完全没有编程经验的小白
12                      真的真的不建议买
Name: 内容, dtype: object

In [19]:
# 截取评判标准  1-好评；0差评
data.loc[data.loc[:,'评价'] == '好评','评论标号'] = 1
data.loc[data.loc[:,'评价'] == '差评','评论标号'] = 0

In [21]:
data

,Unnamed: 0,内容,评价,评论标号
0,0,从编程小白的角度看，入门极佳。,好评,1.0
1,1,很好的入门书，简洁全面，适合小白。,好评,1.0
2,2,讲解全面，许多小细节都有顾及，三个小项目受益匪浅。,好评,1.0
3,3,前半部分讲概念深入浅出，要言不烦，很赞,好评,1.0
4,4,看了一遍还是不会写，有个概念而已,差评,0.0
5,5,中规中矩的教科书，零基础的看了依旧看不懂,差评,0.0
6,6,内容太浅显，个人认为不适合有其它语言编程基础的人,差评,0.0
7,7,破书一本,差评,0.0
8,8,适合完完全全的小白读，有其他语言经验的可以去看别的书,差评,0.0
9,9,基础知识写的挺好的！,好评,1.0


In [28]:
# 选择停用词
stopwords=[]
with open('./data/stopwords.txt','r',encoding='utf-8') as f:
    lines=f.readlines()
    # print(lines)
    for tmp in lines:
        line=tmp.strip()
        # print(line)
        stopwords.append(line)

In [29]:
stopwords

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '--',
 '.',
 '..',
 '...',
 '......',
 '...................',
 './',
 '.一',
 '.数',
 '.日',
 '/',
 '//',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '://',
 '::',
 ';',
 '<',
 '=',
 '>',
 '>>',
 '?',
 '@',
 'A',
 'Lex',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'exp',
 'sub',
 'sup',
 '|',
 '}',
 '~',
 '~~~~',
 '·',
 '×',
 '×××',
 'Δ',
 'Ψ',
 'γ',
 'μ',
 'φ',
 'φ．',
 'В',
 '—',
 '——',
 '———',
 '‘',
 '’',
 '’‘',
 '“',
 '”',
 '”，',
 '…',
 '……',
 '…………………………………………………③',
 '′∈',
 '′｜',
 '℃',
 'Ⅲ',
 '↑',
 '→',
 '∈［',
 '∪φ∈',
 '≈',
 '①',
 '②',
 '②ｃ',
 '③',
 '③］',
 '④',
 '⑤',
 '⑥',
 '⑦',
 '⑧',
 '⑨',
 '⑩',
 '──',
 '■',
 '▲',
 '',
 '、',
 '。',
 '〈',
 '〉',
 '《',
 '》',
 '》），',
 '」',
 '『',
 '』',
 '【',
 '】',
 '〔',
 '〕',
 '〕〔',
 '㈧',
 '一',
 '一.',
 '一一',
 '一下',
 '一个',
 '一些',
 '一何',
 '一切',
 '一则',
 '一则通过',
 '一天',
 '一定',
 '一方面',
 '一旦',
 '一时',
 '一来',
 '一样',
 '一次',
 '一片',
 '一番',
 '一直',
 '一致',
 '一般',
 '一起',
 '一转眼'

In [30]:
#对停用词表进行去重
stopwords=list(set(stopwords))#去重  列表形式
print(stopwords)

['', '“', '［②④', '始而', '不仅', '＇', '？', '重要', '［①ｅ］', '不仅仅', '兮', '纵', '来说', '怎么样', '＝″', '咱们', '直到', '｝', '巨大', '④', '反倒', '庶乎', '是的', '都', '梆', '断然', '严重', '即是说', '哪里', '］', '不下', '个', '云尔', '但凡', '充分', '三番两次', '也', '到底', '而论', '必', '就要', '与', '真是', '合理', '多多', '边', '全身心', '有著', '最高', '按', '不要', '惯常', '［④ｂ］', ']', '方能', '本身', '成为', '尽心尽力', '屡次', '次第', '彻夜', '看上去', '相应', '一番', '岂非', '一天', 'exp', '一旦', '与此同时', '冒', '还', '──', '非独', '取道', '虽', '何须', '比照', '共', '奋勇', '战斗', '依据', '3', 'ＬＩ', '就', '传说', '大事', '一样', '到处', '从', '虽然', '但愿', '应用', '不同', 'ｃ］', '除去', '後来', '之所以', '腾', '不管怎样', '有', '八成', '严格', '为', '多年前', '}', '这么些', '进去', '漫说', '累年', '甚么', '会', '最後', '某某', '假如', '将近', '她的', '结合', '［③ｅ］', '汝', '不大', '充其极', '呀', '多年来', '某些', '不可', '尽可能', '不巧', '把', '》），', '应当', '再者说', '进入', '乘', '突出', '以免', '从古到今', '连日来', '如同', '从事', '依靠', '咦', '怎么办', '犹自', '你的', '基本', '逢', '难说', '忽然', '练习', '每时每刻', '不止', '才能', '啐', '随著', '［①ａ］', '这些', '－', '接著', '尽', '曾经', '管', '下面', '⑤', '简言之', '·', '２', '趁早', 'su

In [35]:
commnet_list =[]
for temp in content:
    seg_list  =  jieba.cut(temp)
    seg_str = ','.join(seg_list)  # 拼接字符串
    print(seg_str)
    commnet_list.append(seg_str)

 ,从,编程,小白,的,角度看,，,入门,极佳,。
很,好,的,入门,书,，,简洁,全面,，,适合,小白,。
讲解,全面,，,许多,小,细节,都,有,顾及,，,三个,小,项目,受益匪浅,。
前半部,分讲,概念,深入浅出,，,要言不烦,，,很赞
看,了,一遍,还是,不会,写,，,有个,概念,而已
中规中矩,的,教科书,，,零,基础,的,看,了,依旧,看不懂
内容,太,浅显,，,个人,认为,不,适合,有,其它,语言,编程,基础,的,人
破书,一本
适合,完完全全,的,小白读,，,有,其他,语言,经验,的,可以,去,看,别的,书
基础知识,写,的,挺,好,的,！
太,基础
略,_,嗦,。,。,适合,完全,没有,编程,经验,的,小白
真的,真的,不,建议,买


In [36]:
commnet_list

[' ,从,编程,小白,的,角度看,，,入门,极佳,。',
 '很,好,的,入门,书,，,简洁,全面,，,适合,小白,。',
 '讲解,全面,，,许多,小,细节,都,有,顾及,，,三个,小,项目,受益匪浅,。',
 '前半部,分讲,概念,深入浅出,，,要言不烦,，,很赞',
 '看,了,一遍,还是,不会,写,，,有个,概念,而已',
 '中规中矩,的,教科书,，,零,基础,的,看,了,依旧,看不懂',
 '内容,太,浅显,，,个人,认为,不,适合,有,其它,语言,编程,基础,的,人',
 '破书,一本',
 '适合,完完全全,的,小白读,，,有,其他,语言,经验,的,可以,去,看,别的,书',
 '基础知识,写,的,挺,好,的,！',
 '太,基础',
 '略,_,嗦,。,。,适合,完全,没有,编程,经验,的,小白',
 '真的,真的,不,建议,买']

In [48]:
# 统计词的个数
transfer = CountVectorizer(stop_words=stopwords)
count_data = transfer.fit_transform(commnet_list)
print(transfer.get_feature_names_out())
count_data.toarray()

['一本' '一遍' '三个' '中规中矩' '依旧' '入门' '内容' '分讲' '前半部' '受益匪浅' '基础' '基础知识' '完完全全'
 '小白' '小白读' '建议' '很赞' '教科书' '有个' '极佳' '概念' '浅显' '深入浅出' '看不懂' '真的' '破书'
 '简洁' '细节' '经验' '编程' '要言不烦' '角度看' '讲解' '语言' '适合' '项目' '顾及']


E:\develop\Anaconda-evns\qiankun\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0

In [49]:
good_or_bad = data['评价'].values  # 获取数据

# 2.6）准备训练集和测试集
# 准备训练集   这里将文本前10行当做训练集  后3行当做测试集
x_train = count_data.toarray()[:10, :]
y_train = good_or_bad[:10]
# 准备测试集
x_text = count_data.toarray()[10:, :]
y_text = good_or_bad[10:]

# 模型训练

In [50]:
# 构建贝叶斯算法分类器
mb = MultinomialNB(alpha=1)  # alpha 为可选项，默认 1.0，添加拉普拉修/Lidstone 平滑参数
# 训练数据
mb.fit(x_train, y_train)
# 预测数据
y_predict = mb.predict(x_text)
#预测值与真实值展示
print('预测值：',y_predict)
print('真实值：',y_text)

预测值： ['差评' '差评' '差评']
真实值： ['差评' '差评' '差评']


# 模型评估

In [51]:
mb.score(x_text, y_text)

1.0